In [3]:
import numpy as np
import onnx
import onnxruntime as rt
import time
import tvm 
import torch
from tvm import relay
from tvm.contrib import graph_runtime
from transformers import BertTokenizer, BertModel, TFBertModel

In [15]:
batch, seq = 1, 128 
N = 1000
pt_model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
t1 = time.time()
for _ in range(N):
    feed_dict = { 
        'input_ids' : torch.tensor(np.random.randint(0, 3e4, size=[batch,seq]).astype("int64")),
        'attention_mask' : torch.tensor(np.zeros([batch,seq]).astype("int64")),
        'token_type_ids' : torch.tensor(np.zeros([batch,seq]).astype("int64"))
    }
    res = pt_model(**feed_dict)
t2 = time.time()
print("pt=", (t2 - t1)/N*1000)

pt= 47.01161861419678


In [17]:
pt_model.eval()

# If you are instantiating the model with `from_pretrained` you can also easily set the TorchScript flag
pt_model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Creating the trace

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'model' is not defined

In [12]:
BertModel.from_pretrained?

In [16]:
traced_model = torch.jit.trace(pt_model, [feed_dict['input_ids'], feed_dict['attention_mask'], feed_dict['token_type_ids']])

In [18]:
torch.jit.save(traced_model, "traced_bert.pt")

In [19]:
loaded_model = torch.jit.load("traced_bert.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=BertModel
  (embeddings): RecursiveScriptModule(
    original_name=BertEmbeddings
    (word_embeddings): RecursiveScriptModule(original_name=Embedding)
    (position_embeddings): RecursiveScriptModule(original_name=Embedding)
    (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
    (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
    (dropout): RecursiveScriptModule(original_name=Dropout)
  )
  (encoder): RecursiveScriptModule(
    original_name=BertEncoder
    (layer): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=BertLayer
        (attention): RecursiveScriptModule(
          original_name=BertAttention
          (self): RecursiveScriptModule(
            original_name=BertSelfAttention
            (query): RecursiveScriptModule(original_name=Linear)
            (key): RecursiveScriptModule(original_name=Linear)
            (value): RecursiveScriptM

In [25]:
with torch.no_grad():
    res = loaded_model(input_ids=feed_dict['input_ids'], attention_mask=feed_dict['attention_mask'], input=feed_dict['token_type_ids'])

In [52]:
batch, seq = 1, 8
t1 = time.time()
with torch.no_grad():
    for _ in range(N):
        feed_dict = { 
            'input_ids' : torch.tensor(np.random.randint(0, 3e4, size=[batch,seq]).astype("int64")),
            'attention_mask' : torch.tensor(np.zeros([batch,seq]).astype("int64")),
            'token_type_ids' : torch.tensor(np.zeros([batch,seq]).astype("int64"))
        }
        res = loaded_model(input_ids=feed_dict['input_ids'], attention_mask=feed_dict['attention_mask'], input=feed_dict['token_type_ids'])
t2 = time.time()
print("pt=", (t2 - t1)/N*1000)

pt= 11.830577611923218


In [34]:
loaded_model(feed_dict['input_ids'], feed_dict['attention_mask'], )

RuntimeError: forward() is missing value for argument 'input'. Declaration: forward(__torch__.transformers.models.bert.modeling_bert.___torch_mangle_415.BertModel self, Tensor input_ids, Tensor attention_mask, Tensor input) -> ((Tensor, Tensor))

In [38]:
traced_model = torch.jit.trace(pt_model,example_inputs=[torch.tensor(np.random.randint(0, 3e4, size=[batch,seq]).astype("int64")),torch.tensor(np.zeros([batch,seq]).astype("int64"))])

In [39]:
traced_model(feed_dict['input_ids'], feed_dict['attention_mask'], )

(tensor([[[ 1.6880e-01,  2.0750e-01,  3.0610e-01,  ..., -3.3834e-01,
           -2.9114e-03, -2.6709e-01],
          [ 8.2824e-02,  3.9966e-01,  3.7773e-01,  ..., -4.8141e-01,
            2.1363e-02, -1.8712e-01],
          [ 2.1411e-02,  3.7622e-01,  4.3044e-01,  ..., -5.0092e-01,
            7.8786e-02,  2.2360e-03],
          ...,
          [-1.4268e-01,  2.9536e-01,  3.6935e-01,  ..., -3.9757e-01,
            2.8217e-02, -7.9942e-05],
          [-1.0709e-01,  3.5698e-01,  3.9346e-01,  ..., -1.5653e-01,
            9.1324e-02, -2.1205e-02],
          [-9.7605e-03,  3.1167e-01,  3.5784e-01,  ..., -3.7722e-01,
            2.6488e-02,  1.1149e-01]]], grad_fn=<NativeLayerNormBackward>),
 tensor([[-0.7819, -0.1935,  0.9130,  0.6324, -0.6431, -0.0818,  0.7254,  0.2976,
           0.9038, -0.9974,  0.8257, -0.7815,  0.9843, -0.6926,  0.9717, -0.2382,
           0.4736, -0.4750,  0.4130, -0.6882,  0.8582, -0.6545,  0.7674,  0.2558,
           0.2403, -0.9308, -0.0749,  0.9681,  0.9548,  0.8

In [46]:
for p in traced_model.parameters():
    p.requires_grad_(False)

In [57]:
batch, seq = 1, 128
shape = {
    "input_ids" : (batch, seq),
    "attention_mask" : (batch, seq),
}

feed_dict = {
    'input_ids' : torch.tensor(np.random.randint(0, 10000, size=[batch,seq]).astype("int64")),
    'attention_mask' : torch.tensor(np.zeros([batch,seq]).astype("int64")),
}
shape["token_type_ids"] = (batch, seq)
feed_dict["token_type_ids"] = torch.tensor(np.zeros([batch,seq]).astype("int64"))

In [54]:
mod, params = relay.frontend.from_pytorch?

In [65]:
mod, params = relay.frontend.from_pytorch(loaded_model, [(k,v) for k,v in shape.items()],default_dtype="float32")